In [1]:
from keras.datasets import fashion_mnist
from art.utils import load_mnist
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Lambda, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential,load_model
from art.metrics.metrics import empirical_robustness, clever_t, clever_u, clever, loss_sensitivity, wasserstein_distance
import numpy as np
from art.estimators.classification.keras import KerasClassifier
import logging
from progressbar import ProgressBar

import sys
sys.path.append('..')  
from GradPri_utils.utils import *

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
tf.compat.v1.disable_eager_execution()

In [2]:
R_L1 = 40
R_L2 = 2
R_LI = 0.1

def _create_krclassifier_FMNIST():
    num_classes = 10
    input_shape = (28,28,1)
    
    model = Sequential()
    # Block 1
    model.add(Conv2D(64, (3, 3), padding='same', name = 'block1_conv1', 
                     input_shape=input_shape))
    # model.add(BatchNormalization(name = 'block1_bn1'))
    model.add(Activation('relu', name = 'block1_act1'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block1_pool1'))

    model.add(Conv2D(128, (3, 3), padding='same', name = 'block1_conv2'))
    # model.add(BatchNormalization(name = 'block1_bn2'))
    model.add(Activation('relu', name = 'block1_act2'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block1_pool2'))

    model.add(Conv2D(256, (3, 3), padding='same', name = 'block1_conv3'))
    # model.add(BatchNormalization(name = 'block1_bn3'))
    model.add(Activation('relu', name = 'block1_act3'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block1_pool3'))
    # top
    model.add(Flatten(name='flatten'))
    model.add(Dense(1024, activation='relu', name='fc1'))
    model.add(Dropout(0.5, name='drop1'))
    model.add(Dense(32, activation='relu', name='fc2'))
    model.add(Dropout(0.5, name='drop2'))
    # model.add(Dense(64, activation='relu', name='fc3'))
    # model.add(Dropout(0.5, name='drop3'))
    model.add(Dense(num_classes, activation='softmax', name='predictions'))

    
    # model = load_model("./fmnist_CNN.h5")
    opt = tf.keras.optimizers.Adam(learning_rate=0.00005)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

    # Get the classifier
    krc = KerasClassifier(model=model, clip_values=(0, 1), use_logits=False)

    return krc

logger = logging.getLogger(__name__)

In [3]:
# Get Fashion-MNIST
(x_train, y_train), (x_test, y_test), _, _ = load_mnist()

# Get the classifier
# krc = _create_krclassifier_FMNIST()
# krc.fit(x_train, y_train, batch_size=256, nb_epochs=5, shuffle=True,
#         validation_data=(x_test[0:5000], y_test[0:5000]), verbose=1)

krc = load_model('./fmnist_CNN_art.h5')
krc = KerasClassifier(model=krc, clip_values=(0, 1), use_logits=False)

In [4]:
# krc.save(filename='fmnist_CNN_art.h5', path='.')

In [4]:
# Gini = []
# total_sample_num = 2000
# predicted_confidence = []

# pbar = ProgressBar()
# for index in pbar(range(total_sample_num)):
#     pre_tmp = krc.predict(np.reshape(x_test[index], [-1,28,28,1]))
#     Gini.append(1-np.sum(pre_tmp*pre_tmp))
#     predicted_confidence.append(pre_tmp)

/public/liujiawei/anaconda3/envs/ZHB_env/lib/python3.6/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
100% |########################################################################|


In [5]:
# indexs = np.argsort(Gini)
# indexs = indexs[::-1]
# # 计算APFD指标
# APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=y_test[range(total_sample_num)],
#                    predicted_confidence=np.array(predicted_confidence))
# print("APFD: ", APFD)

# # 计算RAUC指标
# RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=y_test[range(total_sample_num)],
#                     predicted_confidence=np.array(predicted_confidence))
# print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.9419964788732395
RAUC:  0.9585205891839587


In [4]:
Gini = []
total_sample_num = 2000
predicted_confidence = []

pbar = ProgressBar()
for index in pbar(range(total_sample_num)):
    pre_tmp = krc.predict(np.reshape(x_test[index], [-1,28,28,1]))
    Gini.append(
        np.sqrt(
            ( np.sum( (np.max(pre_tmp)-pre_tmp) * (np.max(pre_tmp)-pre_tmp) ) ) / len(pre_tmp[0])
        )
    )
    predicted_confidence.append(pre_tmp)

/public/liujiawei/anaconda3/envs/ZHB_env/lib/python3.6/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
100% |########################################################################|


In [5]:
indexs = np.argsort(Gini)
# indexs = indexs[::-1]
# 计算APFD指标
APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=y_test[range(total_sample_num)],
                   predicted_confidence=np.array(predicted_confidence))
print("APFD: ", APFD)

# 计算RAUC指标
RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=y_test[range(total_sample_num)],
                    predicted_confidence=np.array(predicted_confidence))
print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.942637323943662
RAUC:  0.9591728487976203


In [7]:
# craft adversarial examples
x_adv = []
y_adv = []
max_iter = 100
eps = 50
thre_conf = 0.5# 0.999999
pbar = ProgressBar()
for i in pbar(range(100)):
    pre_tmp = krc.predict(np.reshape(x_test[i], [-1,28,28,1]))
    x_adv_tmp = np.copy(x_test[i])
    # check whether x is right prediction
    if np.argmax(pre_tmp[0]) == np.argmax(y_test[i]):
        for j in range(max_iter):
            grad_tmp = krc.class_gradient(np.reshape(x_adv_tmp, [-1,28,28,1]), 
                                                 label=np.argmax(y_test[i]))[0][0]
            x_adv_tmp = x_adv_tmp - eps * grad_tmp
            pre_adv_tmp = krc.predict(np.reshape(x_adv_tmp, [-1,28,28,1]))
            if np.argmax(pre_adv_tmp[0]) != np.argmax(y_test[i]):
                if np.max(pre_adv_tmp[0])>thre_conf:
                    x_adv.append(x_adv_tmp)
                    y_adv.append(y_test[i])
                    break

In [8]:
# x_adv = np.array(x_adv)
# y_adv = np.array(y_adv)
# np.save('./x_adv.npy', x_adv)
# np.save('./y_adv.npy', y_adv)
x_adv = np.load('./x_adv.npy')
y_adv = np.load('./y_adv.npy')
print(x_adv.shape)

(15, 28, 28, 1)


In [9]:
X_adv_clean = np.concatenate((x_adv, x_test[100:250]))
Y_adv_clean = np.concatenate((y_adv, y_test[100:250]))

In [10]:
Gini_adv = []
predicted_confidence = []

pbar = ProgressBar()
for index in pbar(range(len(X_adv_clean))):
    pre_tmp = krc.predict(np.reshape(X_adv_clean[index], [-1,28,28,1]))
#     Gini_adv.append(
#         np.sqrt(
#             ( np.sum( (np.max(pre_tmp)-pre_tmp) * (np.max(pre_tmp)-pre_tmp) ) ) / len(pre_tmp[0])
#         )
#     )
    Gini_adv.append(1-np.sum(pre_tmp*pre_tmp))
    predicted_confidence.append(pre_tmp)

100% |########################################################################|


In [11]:
indexs = np.argsort(Gini_adv)
indexs = indexs[::-1]
# 计算APFD指标
APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=Y_adv_clean,
                   predicted_confidence=np.array(predicted_confidence))
print("APFD: ", APFD)

# 计算RAUC指标
RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=Y_adv_clean,
                    predicted_confidence=np.array(predicted_confidence))
print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.20673400673400674
RAUC:  0.21476748313809016


In [28]:
# BoundaryGini = []
# predicted_confidence = []

# R_L1 = 40
# R_L2 = 2
# R_LI = 0.1
# pbar = ProgressBar()
# for index in pbar(range(len(X_adv_clean))):
#     pre_tmp = krc.predict(np.reshape(X_adv_clean[index], [-1,28,28,1]))
#     res_tmp = clever_u(krc, X_adv_clean[index], 10, 10, R_L2, norm=2, pool_factor=5, verbose=False)
#     BoundaryGini.append(np.min([res_tmp, 2]))
#     predicted_confidence.append(pre_tmp)

100% |########################################################################|


In [29]:
# indexs = np.argsort(BoundaryGini)
# # indexs = indexs[::-1]
# # 计算APFD指标
# APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=Y_adv_clean,
#                    predicted_confidence=np.array(predicted_confidence))
# print("APFD: ", APFD)

# # 计算RAUC指标
# RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=Y_adv_clean,
#                     predicted_confidence=np.array(predicted_confidence))
# print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.6364102564102564
RAUC:  0.7045977011494253


In [39]:
BoundaryGini = []
predicted_confidence = []

R_L1 = 40
R_L2 = 2
R_LI = 0.1
pbar = ProgressBar()
for index in pbar(range(len(X_adv_clean))):
    pre_tmp = krc.predict(np.reshape(X_adv_clean[index], [-1,28,28,1]))
#     res_tmp = clever_u(krc, X_adv_clean[index], 10, 10, R_L2, norm=2, pool_factor=5, verbose=False)
#     BoundaryGini.append(np.min([res_tmp, 2]))
    res_tmp = clever(krc, X_adv_clean[index], 10, 10, R_L2, norm=2, pool_factor=5, verbose=False)
    BoundaryGini.append(
        np.sqrt(
            ( np.sum( (np.min(res_tmp)-res_tmp) * (np.min(res_tmp)-res_tmp) ) ) / len(res_tmp)
        )
    )
    predicted_confidence.append(pre_tmp)

100% |########################################################################|


In [40]:
indexs = np.argsort(BoundaryGini)
# indexs = indexs[::-1]
# 计算APFD指标
APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=Y_adv_clean,
                   predicted_confidence=np.array(predicted_confidence))
print("APFD: ", APFD)

# 计算RAUC指标
RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=Y_adv_clean,
                    predicted_confidence=np.array(predicted_confidence))
print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.5943589743589743
RAUC:  0.6574712643678161


In [204]:
grad_Gini = []
predicted_confidence = []
pbar = ProgressBar()
for index in pbar(range(len(X_adv_clean))):
    pre_tmp = krc.predict(np.reshape(X_adv_clean[index], [-1,28,28,1]))
    grad_tmp = krc.class_gradient(np.reshape(X_adv_clean[index], [-1,28,28,1]), 
                                  label=np.argmax(pre_tmp[0]))[0][0]
    grad_Gini.append(np.sum(np.abs(grad_tmp)))
    predicted_confidence.append(pre_tmp)

100% |########################################################################|


In [205]:
indexs = np.argsort(grad_Gini)
indexs = indexs[::-1]
# 计算APFD指标
APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=Y_adv_clean,
                   predicted_confidence=np.array(predicted_confidence))
print("APFD: ", APFD)

# 计算RAUC指标
RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=Y_adv_clean,
                    predicted_confidence=np.array(predicted_confidence))
print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.13076923076923078
RAUC:  0.13793103448275862


In [13]:
R_L1 = 40
R_L2 = 2
R_LI = 0.05
BoundaryGini2 = []

pbar = ProgressBar()
for index in pbar(range(len(X_adv_clean))):
    pre_tmp = krc.predict(np.reshape(X_adv_clean[index], [-1,28,28,1]))
    res_tmp = inverper_c(krc, X_adv_clean[index], 10, 10, R_L2, norm=np.inf, pool_factor=5, prob_constant=0.5)
    BoundaryGini2.append(res_tmp)

100% |########################################################################|


In [14]:
indexs = np.argsort(BoundaryGini2)
# indexs = np.argsort(np.abs(BoundaryGini2))
# indexs = indexs[::-1]
# 计算APFD指标
APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=y_test[range(total_sample_num)],
                   predicted_confidence=np.array(predicted_confidence))
print("APFD: ", APFD)

# 计算RAUC指标
RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=y_test[range(total_sample_num)],
                    predicted_confidence=np.array(predicted_confidence))
print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.5086694762191452
RAUC:  0.9270051508462105


In [12]:
from __future__ import absolute_import, division, print_function, unicode_literals

from functools import reduce
import logging
from typing import Any, Dict, List, Optional, Union, TYPE_CHECKING

import numpy as np
import numpy.linalg as la
from scipy.optimize import fmin as scipy_optimizer
from scipy.stats import weibull_min
from tqdm.auto import tqdm

from art.config import ART_NUMPY_DTYPE
from art.attacks.evasion.fast_gradient import FastGradientMethod
from art.attacks.evasion.hop_skip_jump import HopSkipJump
from art.utils import random_sphere

if TYPE_CHECKING:
    from art.attacks.attack import EvasionAttack
    from art.utils import CLASSIFIER_TYPE, CLASSIFIER_LOSS_GRADIENTS_TYPE, CLASSIFIER_CLASS_LOSS_GRADIENTS_TYPE
    
def inverper_c(
    classifier: "CLASSIFIER_CLASS_LOSS_GRADIENTS_TYPE",
    x: np.ndarray,
    nb_batches: int,
    batch_size: int,
    radius: float,
    norm: float,
    c_init: float = 1.0,
    pool_factor: int = 10,
    prob_constant: float = 0.05,
) -> float:
    """
    Compute CLEVER score for a targeted attack.

    | Paper link: https://arxiv.org/abs/1801.10578

    :param classifier: A trained model.
    :param x: One input sample.
    :param nb_batches: Number of repetitions of the estimate.
    :param batch_size: Number of random examples to sample per batch.
    :param radius: Radius of the maximum perturbation.
    :param norm: Current support: 1, 2, np.inf.
    :param c_init: Initialization of Weibull distribution.
    :param pool_factor: The factor to create a pool of random samples with size pool_factor x n_s.
    :return: CLEVER score.
    """
    # Check if the targeted class is different from the predicted class
    y_pred = classifier.predict(np.array([x]))
    pred_class = np.argmax(y_pred, axis=1)[0]

    # Check if pool_factor is smaller than 1
    if pool_factor < 1:  # pragma: no cover
        raise ValueError("The `pool_factor` must be larger than 1.")

    # Some auxiliary vars
    rand_pool_grad_set = []
    grad_norm_set = []
    dim = reduce(lambda x_, y: x_ * y, x.shape, 1)
    shape = [pool_factor * batch_size]
    shape.extend(x.shape)

    # Generate a pool of samples
    rand_pool = np.reshape(
        random_sphere(nb_points=pool_factor * batch_size, nb_dims=dim, radius=radius, norm=norm),
        shape,
    )
    rand_pool += np.repeat(np.array([x]), pool_factor * batch_size, 0)
    rand_pool = rand_pool.astype(ART_NUMPY_DTYPE)
    if hasattr(classifier, "clip_values") and classifier.clip_values is not None:
        np.clip(rand_pool, classifier.clip_values[0], classifier.clip_values[1], out=rand_pool)

    # Change norm since q = p / (p-1)
    if norm == 1:
        norm = np.inf
    elif norm == np.inf:
        norm = 1
    elif norm != 2:  # pragma: no cover
        raise ValueError(f"Norm {norm} not supported")

    # Compute gradients for all samples in rand_pool
    for i in range(batch_size):
        rand_pool_batch = rand_pool[i * pool_factor : (i + 1) * pool_factor]

        # Compute gradients
        grad_pred_class = classifier.class_gradient(rand_pool_batch, label=pred_class)

        if np.isnan(grad_pred_class).any() :  # pragma: no cover
            raise Exception("The classifier results NaN gradients.")

        grad = grad_pred_class
        grad = np.reshape(grad, (pool_factor, -1))
        grad = np.linalg.norm(grad, ord=norm, axis=1)
        rand_pool_grad_set.extend(grad)

    rand_pool_grads = np.array(rand_pool_grad_set)

    # Loop over the batches
    for _ in range(nb_batches):
        # Random selection of gradients
        grad_norm = rand_pool_grads[np.random.choice(pool_factor * batch_size, batch_size)]
        grad_norm = np.max(grad_norm)
        grad_norm_set.append(grad_norm)

    # Maximum likelihood estimation for max gradient norms
    [_, loc, _] = weibull_min.fit(-np.array(grad_norm_set), c_init, optimizer=scipy_optimizer)

    # Compute function value
    values = classifier.predict(np.array([x]))
#     value = values[:, pred_class] - prob_constant
    value = values[:, pred_class] * np.log(1+values[:, pred_class])

    # Compute scores
#     score = np.min([-value[0] / loc, radius])
    score = -value[0] / loc

    return score
